# Energy Mix
**Group 4**  
Jianyi Chen, Jincheng Hong, Tianyu Su, Yicheng Huang

## 1. Problem Statement

Construct a mathematical model predicting the compressive strength using features including: cement component, water component, coarse aggregate component and the fine aggregate component following the linear regression model.

## 2. Variables and Parameters  

**Task:** Complete the table parameters and variables.

Hint: It may be helpful to know that GPa is gigapascals, which is a unit of pressure.  One megapascal is equal to one billion pascals and a Pascal in scientific units is: Pa $ = \frac{\textrm{kg}}{\textrm{m s}}.$


| Description | Symbol | Units | Type|
|-------------|--------|-------|-------|
| Concrete Compressive Strength | $Y$ | GPa | target (dependent) variable |
| Cement Component | $X1$ | kg/m$^3$| feature (independent) variable |
| Water Component | $X2$ | - | feature (independent) variable |
| Coarse Aggregate Component | $X3$ | kg/m$^3$| feature (independent) variable |
| Fine Aggregate Component | $X4$ | kg/m$^3$| feature (independent) variable |
| regression coefficent for intercept | $\beta_0$ | GPa | parameter |
| regression coefficent for Cement Component | $\beta_1$ | m$^2$/s | parameter |
| regression coefficent for Water Component | $\beta_2$ | kg/(m s) | parameter |
| regression coefficent for Coarse Aggregate Component | $\beta_3$ | m$^2$/s | parameter |
| regression coefficent for Fine Aggregate Component | $\beta_4$ | m$^2$/s | parameter |
| residural | $\epsilon$ | GPa | parameter |


## 3. Assumptions and Constraints

**Task:** State assumptions and constraints. 

- The model follows the linear model $Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + \beta_4 X_4 + \varepsilon$
- The average value of the error is 0: i.e., $\mathbb{E}(\epsilon_i) = 0$ for all $i$
- The variance of the error is constant: i.e., $Var(\epsilon_i) = \sigma^2$ for all $i$.
- The error $\vec{\epsilon}$ is a random sample from the normal distribution of zero mean and variance $\sigma^2$.
- The error is independent for each $\epsilon_i$ for all $i$.
- The features are independent of each other.
- The features are representative of the compressive strength.

## 4. Build Solutions

**Task:** Construct and run the linear regression model where `concrete_compressive_strength` is the output and `cement`,`water`, `course_aggregate`, and `fine_aggregate` are the inputs.

## 5. Simple Model

In [1]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import pandas as pd

# Load parameters from CSV
parameters_df = pd.read_csv("energy_parameters.csv", index_col=0)

# Define energy sources
energy_sources = parameters_df.index.tolist()

# Extract parameters from CSV
c = parameters_df["LCOE Cost ($/MWh)"].to_dict()
p = parameters_df["Pollution Cost (g/kWh CO2eq)"].to_dict()
f = parameters_df["Fixed Cost ($ per MW)"].to_dict()
l = parameters_df["Production Limit (MW)"].to_dict()
r = parameters_df["Renewable Indicator"].to_dict()
consumption_rate = parameters_df["Consumption Rate"].to_dict()

D = 45263.2  # Total energy demand (MW)
B = 9.36e9  # Total budget ($)
carbon_tax = 10  # Carbon tax ($ per tonne CO2eq)
R_min = 0.8  # Updated renewable energy target to 0.8

# Define variables
x = LpVariable.dicts("x", energy_sources, lowBound=0)  # Energy supply
y = LpVariable.dicts("y", energy_sources, lowBound=0)  # Energy extraction amount

# Create optimization model
model = LpProblem("Energy_Optimization", LpMinimize)

# Objective function: Minimize pollution cost including carbon tax
model += lpSum((p[i] / 1e6) * carbon_tax * x[i] for i in energy_sources)

# Constraint 1: Ensure energy demand is met (considering consumption rate)
model += lpSum(y[i] for i in energy_sources) >= D

# Constraint 2: Production limits
for i in energy_sources:
    model += x[i] <= l[i]
    model += y[i] == x[i] * consumption_rate[i]  # Extraction amount = Production * Consumption rate